# Notebook E-tivity 3 CE4021 Task 2

Student name: Sean Meade

Student ID: 10128921

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [2]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

First I created a function for making a list of the words from the string that represents an email. 

In [3]:
def list_of_words_from_email(email):

    """
    Takes in a string email and produces the words of that email as a list of strings.
    """

    # split the email into a list of words
    words = email.split()
    
    return words

Then a function that takes in a list of emails and counts the words in it. This is used to find the number of words in the test data for the given spam or ham emails.

In [4]:
def dict_count_words_in_emails(words, emails):

    """
    Takes in a list or set of words and checks how many words are in the list of emails.
    """

    words_count = {}

    # for every word in our training data
    for word in words:
        # Create an entry in the dictionary
        words_count[word.lower()] = 0
        # Then loop through the emails
        for email in emails:
            # if the word is in the email
            if word.lower() in email:
                # add the word count to its corresponding dictionary entry
                words_count[word.lower()] += email.count(word)
    
    return words_count

Next I use the output of the last function to calculate the probability the email is in that category.

$$ \mathbb{P}(word|S) = \frac{number~of~spam~emails~containing~the~word~+~1}{number~of~spam~emails~+~2} $$

$$ \mathbb{P}(word|H) = \frac{number~of~ham~emails~containing~the~word~+~1}{number~of~ham~emails~+~2} $$

I use Laplace smoothing to account for words in the test emails that are not in the training emails. This is done by adding 1 to the numerator and 2 to the denominator. So instead of an untrained words working out to a probability of zero it will work out at 1 over the number of emails in that category.

In [5]:
def prob_of_word_in_emails(word_count, emails):

    """
    Calculates the probability of each word given the email is spam or ham.
    """

    # a dictionary that will contain the probabilities for each word for a given category
    word_prob = {}
    # number of emails
    email_count = len(emails)
    # using the word and it's count
    for word, count in word_count.items():
        # calculate the probability that the word is spam or hame
        word_prob[word] = (count + 1)/(email_count + 2)
    
    return word_prob

Then work out the probability of an email being spam or ham based on the training data.

$$ \mathbb{P}(S) = \frac{number~of~spam~emails}{total~number~of~emails} $$
$$ \mathbb{P}(H) = \frac{number~of~ham~emails}{total~number~of~emails} $$

In [6]:
def probability_of_spam_and_ham(spam_emails, ham_emails):
    """
    This function takes the spam and ham emails and returns the probability that an email will be ham or spam based on that
    """

    # a dictionary to hold the returned probabilities
    probs = {}

    # total number of emails
    total_email_count = len(spam_emails) + len(ham_emails)

    # then work out the probabilities by the number of emails in the category divided by the total number of emails 
    probs["prob_email_spam"] = len(spam_emails)/total_email_count
    probs["prob_email_ham"] = len(ham_emails)/total_email_count

    return probs

# Calculate and save the probability an email is spam and ham
prob_of_spam_and_ham = probability_of_spam_and_ham(previous_spam, previous_ham)
P_S = prob_of_spam_and_ham["prob_email_spam"]
P_H = prob_of_spam_and_ham["prob_email_ham"]

And as expected the probability of an email being spam plus the probability of an email being ham is equal to 1, because they are the only two possible outcomes.

In [7]:
P_S + P_H

1.0

This next one uses the list_of_words_from_email function on a list of emails.

In [8]:
def list_of_words_from_emails(emails):
    """
    takes in a list of strings representing emails and returns a list of all the words
    """

    # a list to contain the returned list of words
    words_in_emails = []

    # loop through emails
    for email in emails:
        # split the email into words and add them to returned list
        words_in_emails += list_of_words_from_email(email)

    return words_in_emails

It's important to remove untrained words from the test emails. The words have no weight in the final calculation.

In [9]:
def remove_untrained_words(email, trained_words):

    """
    Takes in a string representing an email and a list of the words used in training. It returns the list of words in the email that are also in the training data.
    """

    # Set containing the unique words of the email
    set_of_email_words = set(list_of_words_from_email(email))

    # list that returns the words in both email and trained_words
    cross_over_words = []

    # for every unique word of the email
    for word in set_of_email_words:
        # if that for is in the training data
        if word in trained_words:
            # add that word to the returned list
            cross_over_words.append(word.lower())

    return cross_over_words

Then tying it all together into one function that returns the probability of each word for that category.

In [10]:
def prob_of_words_for(emails):
    """
    Takes in a list of emails (either ham or spam) and returns the probability of 
    each word given the email was spam or ham.
    """

    words = list_of_words_from_emails(emails)
    word_count = dict_count_words_in_emails(words, emails)
    word_probs = prob_of_word_in_emails(word_count, emails)

    return word_probs

Finally we tie everything together we can work out the probability an email is spam given the words on the email:

$$ \mathbb{P}(S|x_1,...,x_i) \approx \frac{\mathbb{P}(S)\prod_{i=1}^{n}\mathbb{P}(x_i|S)}{\mathbb{P}(S)\prod_{i=1}^{n}\mathbb{P}(x_i|S)~+~\mathbb{P}(H)\prod_{i=1}^{n}\mathbb{P}(x_i|H)} $$


In [24]:
def prob_of_email_being_spam(previous_spam, previous_ham, test_emails):

    categories = ["spam", "ham"]

    email_is_spam = {"spam": {}, "ham": {}}

    # all unique words in the training data
    training_words = set(list_of_words_from_emails(previous_ham) + list_of_words_from_emails(previous_spam))

    prob_of_spam_and_ham = probability_of_spam_and_ham(previous_spam, previous_ham)
    P_S = prob_of_spam_and_ham["prob_email_spam"]
    P_H = prob_of_spam_and_ham["prob_email_ham"]

    # Dictionaries of the probability of each word given the category
    P_w_S = prob_of_words_for(previous_spam)
    P_w_H = prob_of_words_for(previous_ham)

    print(P_w_H)
    print(P_w_S)

    for cat in categories:
        for email in test_emails[cat]:
            words_in_email = set(list_of_words_from_email(email))
            P_wi_S = P_S
            P_wi_H = P_H
            for word in words_in_email:
                if word in training_words:
                    try:
                        P_wi_S *= P_w_S[word]
                    except:
                        pass
                    try:
                        P_wi_H *= P_w_H[word]
                    except:
                        pass
                else:
                    P_wi_S *= 1/(2 + len(previous_spam))
                    P_wi_H *= 1/(2 + len(previous_ham))
            print("P_wi_S", P_wi_S)
            print("P_wi_H", P_wi_H)
            probability_its_spam = P_wi_S/(P_wi_S + P_wi_H)
            email_is_spam[cat][email] = [(probability_its_spam > 0.6), probability_its_spam]

    return email_is_spam

prob_of_email_being_spam(previous_spam, previous_ham, new_emails)

{'your': 0.2, 'activity': 0.6, 'report': 0.4, 'benefits': 0.4, 'physical': 0.4, 'the': 0.4, 'importance': 0.4, 'vows': 0.4}
{'send': 0.6666666666666666, 'us': 0.5, 'your': 0.6666666666666666, 'password': 0.5, 'review': 0.3333333333333333, 'our': 0.8333333333333334, 'website': 0.3333333333333333, 'account': 0.3333333333333333}
P_wi_S 0.031746031746031744
P_wi_H 0.017142857142857144
P_wi_S 0.06349206349206349
P_wi_H 0.006857142857142858
P_wi_S 0.026455026455026454
P_wi_H 0.03428571428571429
P_wi_S 0.09523809523809523
P_wi_H 0.003291428571428572


{'spam': {'renew your password': [True, 0.6493506493506493],
  'renew your vows': [True, 0.9025270758122744]},
 'ham': {'benefits of our account': [False, 0.4355400696864111],
  'the importance of physical activity': [True, 0.9665944942777606]}}

In [21]:
print(previous_ham, previous_spam)

['Your activity report', 'benefits physical activity', 'the importance vows'] ['send us your password', 'review our website', 'send your password', 'send us your account']


The last printed out shows whether the spam filter categorizes it as spam or not. You will notice that the spam emails and the first ham email are predicted correctly. 

<hr style=\"border:2px solid gray\"> </hr>

## Reflection